_This notebook is part of the material for the ["An Introduction to Neural Networks"](https://indico.cism.ucl.ac.be/event/106/) session of the [2021 CISM/CÉCI trainings](https://indico.cism.ucl.ac.be/category/6/)._

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ramonpeter/cism-nn2021/blob/main/cnn/solutions/convolutional_neural_network.ipynb)

# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing

### Load the dataset

In [ ]:
!curl https://www.dropbox.com/s/ww4yar8lm77pt2g/catsanddogs.zip?dl=1 -L -o catsanddogs.zip
!unzip -q catsanddogs.zip 
!rm catsanddogs.zip

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

# Convolution and Max-Pooling
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# 2nd Convolution and Max-Pooling
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# 2nd Convolution and Max-Pooling
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Flatten
cnn.add(tf.keras.layers.Flatten())
# Fully connected
cnn.add(tf.keras.layers.Dense(units=16, activation='relu'))
# Output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
print(cnn.summary())


## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)/255.0 # <- scaling important!
test_image = np.expand_dims(test_image, axis = 0)

In [ ]:
result = cnn.predict(test_image)
if result[0][0] > 0.5:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)